# Welcome to OT as Plate filler script 

start with choosing the number of plates that you want to run and then continue on to setting up the deck. 

FIND SCRIPT ON ROBOT OT028 IN FOLDER Vilhelms_sandbox

In [5]:
## insures that robot is correctly initialized
!systemctl stop opentrons-robot-server

'systemctl' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
# import the script you wnat to run 
import sys
import pandas as pd
import os, shutil
# from datetime import datetime
from IPython.display import Markdown as md
# ## start email and camera functions
import subprocess
import smtplib
import datetime
from datetime import date
import os
from time import sleep
# # Import opentrons.execute
import opentrons.execute


ModuleNotFoundError: No module named 'opentrons'

# Initiate Robot 

In [ ]:

# import datetime
from opentrons import types
from time import sleep


def camera(name_of_image, protocol): 
    """ Take picture with the OT camera, runs bash command 
    input: users name of image
    output: image in directory 
    """

    # camerapath = '/dev/video0'
    # camerapath = '/dev/video1'
    camerapath = '/dev/video2'
    subprocess.check_call(['ffmpeg', '-y', '-f', 'video4linux2', '-s',
                       '640x480', '-i', camerapath, '-ss', '0:0:1', 
                       '-frames', '1', os.path.abspath('.') + '/pictures/' + name_of_image +'.jpg']) 


    print('Cammera on')

protocol = opentrons.execute.get_protocol_api('2.11')

protocol.home()


In [ ]:
protocol = opentrons.execute.get_protocol_api('2.11')

##### RUN CODE ON OPENTRONS
p300tips = [protocol.load_labware('opentrons_96_tiprack_300ul', 11)]
p300 = protocol.load_instrument('p300_multi', 'right', tip_racks=p300tips)

fake_plate = protocol.load_labware('agilent_1_reservoir_290ml', 6)

microplate_type = 'nest_96_wellplate_200ul_flat'
microplates = [protocol.load_labware(microplate_type, s) for s in ['4']]

hight = 20
center_location = fake_plate['A1'].top()
combe_move_right = center_location.move(types.Point(x=50, y=0, z=hight))
combe_move_left = center_location.move(types.Point(x=-50, y=0, z=hight))

# Start Run

In [ ]:
# Take picture of end run

## Turn on lights while running run
protocol.set_rail_lights(True)

image_name = 'NoTips'
## LOOP START
for i in range(20):
    p300.pick_up_tip(p300tips[0]['A1'])
    p300.aspirate(150, microplates[0]['A1'].bottom(1))
    p300.move_to(combe_move_right)
    camera(f'{image_name}_{i}', protocol)
    p300.dispense(160, microplates[0]['A1'].bottom(1))
    p300.return_tip(p300tips[0]['A1'])
protocol.home()

## Turn on lights while running run
protocol.set_rail_lights(False)

print('Your protocol is finished well done!')

In [ ]:
# decode output to dict
def output_decode(completed_process):
    output = completed_process.stdout
    output = output.strip()
    output_str = output.decode()
    output_dict = eval(output_str)
    return output_dict

In [ ]:
# detect tip amount and output
def detect_tip(output_dict):
# # 遍历字典中的键值对
    has_tip_in_key = False
    has_full_tips = False
    for key, value in output_dict.items():
        if 'tip' in key and value != 8:
            p300.pause("error")
            has_tip_in_key = True
            # 添加终止程序的逻辑
            print(f"tip: {key}, value: {value}", 'not 8')
        if 'tip' in key and value == 8:
            print(f"tip: {key}, value: {value}")
            has_tip_in_key = True
            has_full_tips = True
    if not has_tip_in_key:
        # 如果所有键中都没有 'tip'，输出特定的消息
        p300.pause("error")
        print("no tips")
    return has_full_tips

In [ ]:

## Turn on lights while running run
protocol.set_rail_lights(False)

print('Your protocol is finished well done!')

In [ ]:
import subprocess
import json

#file_path = 'D:\\thesis\yolov5-master\detect-label.py'
#source = 'D:\\thesis\yolov5-master\\test-image\\test1.jpg'


## Turn on lights while running run
protocol.set_rail_lights(True)

#change image_name
image_name = 'NoTips'

#saved image
source = '/dev/video2/'

#detect file
file_path = 'D:\\thesis\yolov5-master\detect_label_single.py'
arguments = ["--source", source,'--save-txt']

## LOOP START
for i in range(20):
    source_name = f'{image_name}_{i+1}'
    source_path = f'{source}{source_name}'
    #take tips
    p300.pick_up_tip(p300tips[0]['A1'])
    #go to camera
    p300.move_to(combe_move_right)
    camera(source_name, protocol)
    #camera path: camerapath = '/dev/video2'
    completed_process = subprocess.run(["python", file_path] + arguments, capture_output=True, text=True)
    output = json.loads(completed_process.stdout)
    has_full_tips = detect_tip(output)
    if has_full_tips:
        p300.aspirate(150, microplates[0]['A1'].bottom(1))
        p300.move_to(combe_move_right)
        camera(source_name, protocol) #Overwrite the previous image file
        completed_process = subprocess.run(["python", file_path] + arguments, capture_output=True, text=True)
        output = json.loads(completed_process.stdout)
        for key, value in output.items():
            if 'liquid' in key :
                print(f"liquid: {key}, value: {value}")
    p300.dispense(160, microplates[0]['A1'].bottom(1))
    p300.return_tip(p300tips[0]['A1'])
    protocol.home()

## Turn on lights while running run
protocol.set_rail_lights(False)

print('Your protocol is finished well done!')
             

CompletedProcess(args=['python', 'D:\\thesis\\yolov5-master\\detect-label.py', '--source', 'D:\\thesis\\yolov5-master\\test-image\\test1.jpg', '--save-txt'], returncode=0)

In [4]:
import subprocess

file_path = 'D:\\thesis\yolov5-master\detect_label.py'
source = 'D:\thesis\yolov5-master\test-image\test2.jpg'

arguments = ["--source", source,'--save-txt']

completed_process = subprocess.run(["python", file_path] + arguments, capture_output=True, text=True)

output = completed_process.stdout

print(output)  # 处理输出结果

# MY PLAN
# tips no water
# tips with water 
# tips with Red water 
# No tips on pipet
# tipsEmptyResdual

# tips with X(color) water 

HOW TO EXTRACT CODE FROM OT: 
- OPEN WSL OR UBUNTU ON YOUR COMPUTER
- WRITE: sudo scp -i ot2_ssh_key -r root@10.199.253.233:/var/lib/jupyter/notebooks/Vilhelms_sandbox/pictures /mnt/c/Intel/
- GO TO Intel folder on C drive on your computer and find the file there. 